In [1]:
!pip install xgboost
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/4.5 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
import boto3
import s3fs
import tarfile
import joblib
import tempfile
# **IMPORTANT**: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED'] = str(2)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
from time import gmtime, strftime
import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sklearn.preprocessing import MinMaxScaler
#import tensorflow as tf
from pickle import dump, load
import warnings

import random
# def reset_random_seeds():
#     os.environ['PYTHONHASHSEED'] = str(2)
#     tf.random.set_seed(2)
#     np.random.seed(2)
#     random.seed(2)
#     tf.keras.utils.set_random_seed(2024)

# # Make some random data
# reset_random_seeds()

# # Set seed for reproducibility
# np.random.seed(42)
# #tf.random.set_seed(42)
# pd.options.mode.chained_assignment = None

# Import modules
import itertools
import logging
import math
import pickle
from datetime import datetime
from config_module import Config
config = Config()

import ast
import seaborn as sn
import xgboost as xgb
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error,
                             mean_squared_error)
from sklearn.model_selection import (GridSearchCV, TimeSeriesSplit,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
random.seed(12)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml


sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [3]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

In [4]:
def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*")
    
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            # print(datetime_stamp)
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [5]:
def get_latest_s3_file(bucket_name, prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        #print(obj)
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

In [6]:
fs = s3fs.S3FileSystem()
bucket = config.get_value('bucket')
bucket_new = config.get_value('bucket_new')
prefix_output_file = config.get_value('prefix_cleaned_training_file_path')
file_name_prefix = config.get_value('cleaned_training_filename_prefix')
file_path = get_latest_s3_file(bucket,prefix_output_file,file_name_prefix)

old_data = pd.read_excel('s3://'+f'{bucket}/{file_path}')
parts_list = old_data['part_number'].unique().tolist()

prefix_inp = config.get_value('prefix_inp')
last_tr_date = (pd.to_datetime(old_data['period']).max().strftime('%Y-%m-%d'))

In [7]:
old_data.tail()

,period,part_number,Count_of_order,Total_qty_per_month
264555,2024-02-01,PC1860K1Q3,1.0,33.0
264556,2024-03-01,PC1860K1Q3,1.0,34.0
264557,2024-04-01,PC1860K1Q3,1.0,16.0
264558,2024-05-01,PC1860K1Q3,1.0,25.0
264559,2024-06-01,PC1860K1Q3,1.0,31.0


In [8]:
prefix_bestfit_file=config.get_value('prefix_bestfit_file')
prefix_bestfit_file=prefix_bestfit_file+'/'
file_name_bestfit =config.get_value('file_name_bestfit')
latest_best_fit_data_url = get_latest_s3_file(bucket_new,prefix_bestfit_file, file_name_bestfit)
print(latest_best_fit_data_url)
latest_best_fit_data = pd.read_excel('s3://'+f'{bucket_new}/{latest_best_fit_data_url}')

curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_2024-11-03.xlsx


In [9]:
print(prefix_bestfit_file)
print(file_name_bestfit)

curated-bo/curated-bo-demand-forecast/bestfit-selected-models/
bestfit_selected_models_historic_months_


In [10]:
# latest_best_fit_data['Model Name'].unique()

In [11]:
xgb_parts = latest_best_fit_data[latest_best_fit_data['model_name']=='XGBoost']['part_number'].unique().tolist()
ridge_parts = latest_best_fit_data[latest_best_fit_data['model_name']=='Ridge']['part_number'].unique().tolist()
lasso_parts = latest_best_fit_data[latest_best_fit_data['model_name']=='Lasso']['part_number'].unique().tolist()
lstm_parts = latest_best_fit_data[latest_best_fit_data['model_name']=='LSTM single model']['part_number'].unique().tolist()
prophet_parts = latest_best_fit_data[latest_best_fit_data['model_name']=='Prophet']['part_number'].unique().tolist()

In [12]:
len(xgb_parts), len(ridge_parts), len(lasso_parts), len(lstm_parts), len(prophet_parts)

(1101, 733, 795, 155, 3741)

In [13]:
def get_consecutive_months(start_date, n_months=3):
    """
    Generate a list of consecutive month start dates.

    Args:
    - start_date (str): Starting date in 'YYYY-MM-DD' format.
    - n_months (int): Number of consecutive months to generate.

    Returns:
    - list: List of consecutive month start dates in 'YYYY-MM-DD' format.
    """
    start_date = pd.to_datetime(start_date)
    month_list = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m-%d') for i in range(1, n_months + 1)]
    return month_list

In [14]:
def create_lagged_features(data, lag, pred_win):
    """
    This function creates lagged features for the given time series data.
    
    Parameters:
    data (pd.Series): The input time series data.
    lag (int): The number of lagged periods to create.
    pred_win (int): The prediction window size.
    
    Returns:
    pd.DataFrame: A DataFrame containing the lagged features.
    """
    
    # Initialize an empty DataFrame with the same index as the input data
    lagged_data = pd.DataFrame(index=data.index)
    
    # Create lagged features by shifting the data by (lag + pred_win - 1) periods
    for i in range(1, lag + 1):
        lagged_data[f'lag_{i}'] = data.shift(i + (pred_win - 1))
    
    return lagged_data

# Function to forecast future values using a model
def forecast(df, model, n_periods=3):
    """
    This function forecasts future values using the provided model.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame containing the time series data.
    model (object): The trained model to be used for forecasting.
    n_periods (int): The number of periods to forecast.
    
    Returns:
    list: A list of forecasted values.
    """
    # Get the last 'n_periods' values from the 'Total_qty_per_month' column
    last_values = df['Total_qty_per_month'][-3:].values
    
    # Initialize an empty list to store the forecasted values
    forecasts = []
    
    # Generate forecasts for the specified number of periods
    for _ in range(n_periods):
        # Extract the features for the model (last 'lag' values)
        features = last_values[-3:]
        
        # Predict the next value using the model
        pred = model.predict(features.reshape(1, -1))[0]
        
        # Append the predicted value to the forecasts list
        forecasts.append(pred)
        
        # Update the last values by appending the predicted value and removing the oldest value
        last_values = np.append(last_values, pred)[1:]
    
    return forecasts

In [15]:
# Ridge lag 1
def ridge_lag1_forecast(clean_data, s3_bucketname, pickle_path_ridge_lag1, last_training_date):
    df1=clean_data[:]
    df1=df1[['part_number','period','Total_qty_per_month']]
    df1['period'] = pd.to_datetime(df1['period'])
    df1=df1.sort_values('period')
    model_folder = pickle_path_ridge_lag1
    future_dates = get_consecutive_months(last_training_date)
    final_metric_lag1_ridge=pd.DataFrame()
    s3_client = boto3.client('s3')
    fs = s3fs.S3FileSystem()
    for i in df1['part_number'].unique():
#         try:
        print(i)
        data= df1[df1['part_number']==i]
        data.drop('part_number',axis=1,inplace=True)
        data['month'] = data['period'].dt.month
        lag = 3
        pred_win=1
        lagged_data = create_lagged_features(data['Total_qty_per_month'], lag,pred_win)
        # Combine lagged features with original data
        data_with_lags = pd.concat([data, lagged_data], axis=1)
        data_with_lags.fillna(data_with_lags.median(), inplace=True)
        # Step 2: Split the Data
        train_size = len(data_with_lags)-3
        train_data = data_with_lags.iloc[:train_size]
        test_data = data_with_lags.iloc[train_size:]
        X_train_scaled = train_data.drop(['period', 'month','Total_qty_per_month'], axis=1)
        X_test_scaled= X_train_scaled[-3:]
        y_train = train_data['Total_qty_per_month']
        
        model_file_name_jlb = f"ridge-t-{i}.joblib"
        s3_model_path_jlb = f"{pickle_path_ridge_lag1}/{model_file_name_jlb}"
        print(s3_model_path_jlb)
        with fs.open(f"s3://{s3_bucketname}/{s3_model_path_jlb}") as fh:
            model = joblib.load(fh)
        # Make predictions
        y_pred = model.predict(X_test_scaled)
        X_pred = model.predict(X_train_scaled)
        # Forecast the next 3 periods
        n_periods = 3
        forecasts = forecast(data_with_lags, model, n_periods)
        X_prediction=X_pred.tolist()
        test_pred= y_pred.tolist()
        Y_prediction=forecasts
        X_Y_pred=X_prediction+test_pred+Y_prediction

        metric=pd.DataFrame()
        dates =pd.DataFrame(data['period'])
        new_date = pd.DataFrame({'period': pd.to_datetime(future_dates)})
        dates = pd.concat([dates, new_date], ignore_index=True)
        metric['period']=dates['period']
        qty = pd.DataFrame(data['Total_qty_per_month'])
        new_qty = pd.DataFrame({'Total_qty_per_month': [0,0,0]})
        qty = pd.concat([qty, new_qty], ignore_index=True)
        metric['Actual_values']=qty['Total_qty_per_month']
        metric['Predicted_values_lag1']= pd.DataFrame(X_Y_pred)
        metric['part_number']=i
        final_metric_lag1_ridge=pd.concat([final_metric_lag1_ridge,metric])
    return final_metric_lag1_ridge

In [16]:
prefix_ridge_pickle = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'ridge'}"

In [17]:
prefix_ridge_pickle

'persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge'

In [18]:
#ridge_parts = parts_list[:]
old_data_ridge = old_data[old_data['part_number'].isin(ridge_parts)]
ridge_data_forecast = ridge_lag1_forecast(old_data_ridge, bucket, prefix_ridge_pickle, last_tr_date).reset_index(drop=True)

52159YP906
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-52159YP906.joblib


710710KW70C2
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-710710KW70C2.joblib


531000KC10
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-531000KC10.joblib
521210K010


persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-521210K010.joblib


732300D100B0
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-732300D100B0.joblib


...

73920BZ070
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-73920BZ070.joblib
65637BZ320
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-65637BZ320.joblib
174300T850


persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-174300T850.joblib
PC3820K01F
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-PC3820K01F.joblib


PC3820K01H
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-PC3820K01H.joblib
451000D841C3
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-451000D841C3.joblib


53088YY010
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-53088YY010.joblib
61061YY010
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-61061YY010.joblib
53112YP590


persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-53112YP590.joblib
521191K902
persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/ridge/ridge-t-521191K902.joblib


#### Saving Ridge forecasted data

In [19]:
prefix_reg_forecast = config.get_value('prefix_model_output_forecast_path')
full_path_ridge_forecast = f"s3://{bucket}/{prefix_reg_forecast}"
file_date_forecasting = pd.to_datetime(config.get_value('model_forecasting_date')).strftime("%Y-%m-%d")
ridge_forecast_file_name = f'{config.get_value("ridge_forecast_file_name")}{file_date_forecasting}.xlsx'
print(f"{full_path_ridge_forecast}/{config.get_value('model_forecasting_date')}/{ridge_forecast_file_name}")
# saving file
ridge_data_forecast.to_excel(f"{full_path_ridge_forecast}/{config.get_value('model_forecasting_date')}/{ridge_forecast_file_name}", index=False)


s3://s3-dx-tdem-ie-nonprod-persist/persist-bo/persist-bo-demand-forecast/model-forecasted-output/20241103/ridge_forecast_data_2024-11-03.xlsx


In [20]:
# Lasso lag 1
def lasso_lag1_forecast(clean_data, s3_bucketname, pickle_path_lasso_lag1, last_training_date):
    df1=clean_data[:]
    df1=df1[['part_number','period','Total_qty_per_month']]
    df1['period'] = pd.to_datetime(df1['period'])
    df1=df1.sort_values('period')
    model_folder = pickle_path_lasso_lag1
    future_dates = get_consecutive_months(last_training_date)
    final_metric_lag1_lasso=pd.DataFrame()
    s3_client = boto3.client('s3')
    fs = s3fs.S3FileSystem()
    for i in df1['part_number'].unique():
        try:
            print(i)
            data= df1[df1['part_number']==i]
            data.drop('part_number',axis=1,inplace=True)
            data['month'] = data['period'].dt.month
            lag = 3
            pred_win=1
            lagged_data = create_lagged_features(data['Total_qty_per_month'], lag,pred_win)
            
            # Combine lagged features with original data
            data_with_lags = pd.concat([data, lagged_data], axis=1)
            data_with_lags.fillna(data_with_lags.median(), inplace=True)
            
            # Step 2: Split the Data
            train_size = len(data_with_lags)-3
            train_data = data_with_lags.iloc[:train_size]
            test_data = data_with_lags.iloc[train_size:]
            X_train_scaled = train_data.drop(['period', 'month','Total_qty_per_month'], axis=1)
            X_test_scaled= X_train_scaled[-3:]
            y_train = train_data['Total_qty_per_month']
            
            model_file_name_jlb = f"lasso-t-{i}.joblib"
            s3_model_path_jlb = f"{pickle_path_lasso_lag1}/{model_file_name_jlb}"
            with fs.open(f"s3://{s3_bucketname}/{s3_model_path_jlb}") as fh:
                model = joblib.load(fh)
            
            # Make predictions
            y_pred = model.predict(X_test_scaled)
            X_pred = model.predict(X_train_scaled)
            
            # Forecast the next 3 periods
            n_periods = 3
            forecasts = forecast(data_with_lags, model, n_periods)
            X_prediction=X_pred.tolist()
            test_pred= y_pred.tolist()
            Y_prediction=forecasts
            X_Y_pred=X_prediction+test_pred+Y_prediction
            metric=pd.DataFrame()
            dates =pd.DataFrame(data['period'])
            new_date = pd.DataFrame({'period': pd.to_datetime(future_dates)})
            dates = pd.concat([dates, new_date], ignore_index=True)
            metric['period']=dates['period']
            qty = pd.DataFrame(data['Total_qty_per_month'])
            new_qty = pd.DataFrame({'Total_qty_per_month': [0,0,0]})
            qty = pd.concat([qty, new_qty], ignore_index=True)
            metric['Actual_values']=qty['Total_qty_per_month']
            metric['Predicted_values_lag1']= pd.DataFrame(X_Y_pred)
            metric['part_number']=i
            final_metric_lag1_lasso=pd.concat([final_metric_lag1_lasso,metric])
        except Exception as e:
            print(e)
            print(i)
    return final_metric_lag1_lasso

In [21]:
prefix_lasso_pickle = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'lasso'}"

old_data_lasso = old_data[old_data['part_number'].isin(lasso_parts)]
lasso_data_forecast = lasso_lag1_forecast(old_data_lasso, bucket, prefix_lasso_pickle, last_tr_date).reset_index(drop=True)

52159YP905
815510K360
815510L020


768010K120
690570K200


8481002030
48820F4010


815800K030
48824KK010
815800D500


852140A010
815600K200


...

43420BZ260
166200Y062
52812BZ010


11101B9625
811450KU40


89070BZ550
76822BZ080
9094205043


81551BZ490
815500K700


678370K093
451000KE51C3


#### Saving Lasso forecasted data

In [22]:
full_path_lasso_forecast = f"s3://{bucket}/{prefix_reg_forecast}"
lasso_forecast_file_name = f'{config.get_value("lasso_forecast_file_name")}{file_date_forecasting}.xlsx'

# saving file
lasso_data_forecast.to_excel(f"{full_path_lasso_forecast}/{config.get_value('model_forecasting_date')}/{lasso_forecast_file_name}", index=False)


In [23]:
# Xgboost lag 1
def xgboost_lag1_forecast(clean_data, s3_bucketname, pickle_path_xgboost_lag1, last_training_date):
    df1=clean_data[:]
    df1=df1[['part_number','period','Total_qty_per_month']]
    df1['period'] = pd.to_datetime(df1['period'])
    df1=df1.sort_values('period')
    s3_client = boto3.client('s3')
    fs = s3fs.S3FileSystem()
    
    #create a model folder for saving pickle with a changed name
    model_folder = pickle_path_xgboost_lag1
    future_dates = get_consecutive_months(last_training_date)
    final_metric_lag1=pd.DataFrame()
    for i in df1['part_number'].unique():
        try:
            print(i)
            data= df1[df1['part_number']==i]
            data.drop('part_number',axis=1,inplace=True)
            data['month'] = data['period'].dt.month
            lag = 3
            pred_win=1
            lagged_data = create_lagged_features(data['Total_qty_per_month'], lag,pred_win)
            
            # Combine lagged features with original data
            data_with_lags = pd.concat([data, lagged_data], axis=1)
            data_with_lags.fillna(data_with_lags.median(), inplace=True)
            
            # Step 2: Split the Data
            train_size = len(data_with_lags)-3
            train_data = data_with_lags.iloc[:train_size]
            test_data = data_with_lags.iloc[train_size:]
            X_train_scaled = train_data.drop(['period', 'month','Total_qty_per_month'], axis=1)
            X_test_scaled= X_train_scaled[-3:]
            y_train = train_data['Total_qty_per_month']
            model_file_name_jlb = f"xgb-t-{i}.joblib"
            s3_model_path_jlb = f"{pickle_path_xgboost_lag1}/{model_file_name_jlb}"
            with fs.open(f"s3://{s3_bucketname}/{s3_model_path_jlb}") as fh:
                model = joblib.load(fh)
            
            # Make predictions
            y_pred = model.predict(X_test_scaled)
            X_pred = model.predict(X_train_scaled)
            # Forecast the next 3 periods
            n_periods = 3
            forecasts = forecast(data_with_lags, model, n_periods)
            
            X_prediction=X_pred.tolist()
            test_pred= y_pred.tolist()
            Y_prediction=forecasts
            X_Y_pred=X_prediction+test_pred+Y_prediction
            # Inference collection
            metric=pd.DataFrame()
            dates =pd.DataFrame(data['period'])
            new_date = pd.DataFrame({'period': pd.to_datetime(future_dates)})
            dates = pd.concat([dates, new_date], ignore_index=True)
            metric['period']=dates['period']
            qty = pd.DataFrame(data['Total_qty_per_month'])
            new_qty = pd.DataFrame({'Total_qty_per_month': [0,0,0]})
            qty = pd.concat([qty, new_qty], ignore_index=True)
            metric['Actual_values']=qty['Total_qty_per_month']
            metric['Predicted_values_lag1']= pd.DataFrame(X_Y_pred)
            metric['part_number']=i
            final_metric_lag1=pd.concat([final_metric_lag1,metric])
        except Exception as e:
            print(e)
            print(i)
    return final_metric_lag1

In [24]:
prefix_xgb_pickle = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'xgboost'}"
old_data_xgb = old_data[old_data['part_number'].isin(xgb_parts)]
xgb_data_forecast = xgboost_lag1_forecast(old_data_xgb, bucket, prefix_xgb_pickle, last_tr_date).reset_index(drop=True)

521590A090
9046613004


90401T0001
PC4011200856


P44015800500
PC1620D006


PC16226001
PC4660K00A


53087YP040C0
PZ05700009


...

81150YP380
89341BZ200B0


8110A02U30
89994BZ170


8110B02U20
174300T740


288000T341
PS382BY004


PC55712001
PC1860K1Q3


#### Saving XGBoost forecast file

In [25]:
full_path_xgb_forecast = f"s3://{bucket}/{prefix_reg_forecast}"
xgb_forecast_file_name = f'{config.get_value("xgb_forecast_file_name")}{file_date_forecasting}.xlsx'

# saving file
xgb_data_forecast.to_excel(f"{full_path_xgb_forecast}/{config.get_value('model_forecasting_date')}/{xgb_forecast_file_name}", index=False)